# AC209B Final Project
## Module E: Predicting COVID-19 Cases
- Austin Rochon
- Emily Xie
- Mark Lock

<hr style="height:2pt">

## Table of Contents

0. [Introduction](#introduction)
1. [Data Collection and Cleaning](#data)  
    a. [COVID-19 Cases by State](#us_state_covid)
    b. [Google Community Mobility Reports](#google_community_reports)  
    c. [Google Search Data](#google_search_data)  
2. [Modeling](#modeling)

<a id='introduction'></a>
## Introduction

<a id='data'></a>
## Data Collection and Cleaning

<a id='us_state_covid'></a>
## Global COVID-19 Data
We'll start by examining the global COVID-19 data provided by [Johns Hopkins University](https://github.com/CSSEGISandData).